# Stacking and unstacking data

In [1]:
import pandas as pd
from dfply import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').getOrCreate()

## Reshaping data

Two ways

* We can **stack** data into a *tall* format.
* We can **unstack** data into a *long* format.

## (totally real and not at all made-up) Example - Quarterly Auto Sales

**Note** the last four columns are

* same measurements
* same units

#### `pandas`

In [4]:
sales = pd.read_csv("./data/auto_sales.csv")
sales

,Salesperson,Compact,Sedan,SUV,Truck
0,Ann,22,18,15,12
1,Bob,19,12,17,20
2,Yolanda,19,8,32,15
3,Xerxes,12,23,18,9


## Stacking measurements of the same type/units

<img src="./img/stack_in_action.gif" width=600>

We can fix issues with informative column labels by stacking the data with `gather`

## A Stack by any other name ...

The act of stacking similar columns goes by various names.

* JMP and Minitab call this *stack*
* `pandas` calls this *melt*
* Wickham/`tidyr`/`dfply` call this *gather*

I prefer **stack**, primarily because it makes it clear we are *melting*/*gathering* data vertically.

## Stacking data in `pandas` with `gather`

Syntax: `gather(lbl_col_name, val_col_name, cols_to_stack)`

In [5]:
sales_cols = ['Compact', 'Sedan', 'SUV', 'Truck']
sales_stacked = (sales 
                 >> gather("CarType","QrtSales", sales_cols))
sales_stacked >> head

,Salesperson,CarType,QrtSales
0,Ann,Compact,22
1,Bob,Compact,19
2,Yolanda,Compact,19
3,Xerxes,Compact,12
4,Ann,Sedan,18


In [13]:
df.withColumn("_vars_and_vals", explode(_vars_and_vals))

AttributeError: 'DataFrame' object has no attribute 'withColumn'

## Unstacking Data with `spread`

Syntax: `spread(split_by_col, to_split_col)`

In [14]:
(sales_stacked
 >> spread(X.CarType, X.QrtSales))

,Salesperson,_ID,Compact,SUV,Sedan,Truck
0,Ann,0,22,15,18,12
1,Bob,1,19,17,12,20
2,Xerxes,3,12,18,23,9
3,Yolanda,2,19,32,8,15


## Safely working with `gather` and `spread`


We were lucky the last example worked.  Note that 

* `spread` needs a unique column to work properly.  
* `gather` will add a column by setting `add_id=True`

In [17]:
sales_stacked = sales >> gather("CarType","QrtSales", sales_cols, add_id=True)
sales_stacked >> head(5)

,Salesperson,_ID,CarType,QrtSales
0,Ann,0,Compact,22
1,Bob,1,Compact,19
2,Yolanda,2,Compact,19
3,Xerxes,3,Compact,12
4,Ann,0,Sedan,18


In [16]:
sales_stacked >> spread(X.CarType, X.QrtSales) >> head(2)

,Salesperson,_ID,Compact,SUV,Sedan,Truck
0,Ann,0,22,15,18,12
1,Bob,1,19,17,12,20


## Why Stack?

* Perform transformations on many columns.
* Fix problems with the Golden Rule

## Example - Switching Units on All Sales

Suppose your manager wants these numbers in *monthly* sales.  You could

1. Adjust each column with a separate formula
2. Stack --> Transform once --> Unstack

#### Method 1 - Column Transformations

In [10]:
(sales
 >> mutate(Compact = X.Compact/3,
           SUV =   X.SUV/3,
           Sedan = X.Sedan/3,
           Truck = X.Truck/3)
 >> head(2))

,Salesperson,Compact,Sedan,SUV,Truck
0,Ann,7.333333,6.0,5.000000,4.000000
1,Bob,6.333333,4.0,5.666667,6.666667


#### Method 2 - Stack-Transform-Unstack

In [71]:
(sales 
 >> gather("CarType","QrtSales", sales_cols)
 >> mutate(MonSales = X.QrtSales/3)
 >> drop(X.QrtSales)
 >> spread(X.CarType, X.MonSales)
 >> head(5)
)

,Salesperson,Compact,SUV,Sedan,Truck
0,Ann,7.333333,5.000000,6.000000,4.000000
1,Bob,6.333333,5.666667,4.000000,6.666667
2,Xerxes,4.000000,6.000000,7.666667,3.000000
3,Yolanda,6.333333,10.666667,2.666667,5.000000


## Comparing the two methods

**Method 1:**
* More straight forward
* Lots of repeated code
* Doesn't scale ... imagine 100+ columns

**Method 2:**
* More complicated
* Scales well


## <font color="red"> Exercise 1 </font>
    
**Task:** Load the `health_survey.csv` data and use the Stack-Transform-Unstack trick to transform the responses to a Lickert scale where *Strongly Agree* mapped to 5 and *Strongly Disagree* mapped to 1


In [61]:
survey = pd.read_csv("./data/health_survey.csv")
survey.columns



Index(['person_ID', 'F1', 'F5', 'F2', 'F1.1', 'F2.1', 'F6', 'F4', 'F3', 'F5.1',
       'F1.2', 'F2.2', 'F6.1', 'F2.3', 'F4.1', 'F2.4', 'F5.2', 'F2.5', 'F6.2',
       'F1.3', 'F2.6', 'F5.3', 'F4.2', 'F2.7', 'F3.1', 'F2.8', 'F5.4', 'F3.2',
       'F1.4', 'F3.3', 'F1.5', 'F5.5', 'F6.3', 'F1.6', 'F5.6', 'F2.9', 'F3.4',
       'F4.3', 'F2.10', 'F1.7', 'F6.4', 'F4.4', 'F5.7', 'F3.5', 'F2.11'],
      dtype='object')

In [62]:
likert_dict = {"Strongly Disagree":1,
               "Somewhat Disagree":2,
               "Neither Agree nor Disagree":3,
               "Somewhat Agree":4,
               "Strongly Agree":5}

In [72]:
from more_dfply import recode
questions= ['F1', 'F5', 'F2', 'F1.1', 'F2.1', 'F6', 'F4', 'F3',
       'F5.1', 'F1.2', 'F2.2', 'F6.1', 'F2.3', 'F4.1', 'F2.4', 'F5.2', 'F2.5',
       'F6.2', 'F1.3', 'F2.6', 'F5.3', 'F4.2', 'F2.7', 'F3.1', 'F2.8', 'F5.4',
       'F3.2', 'F1.4', 'F3.3', 'F1.5', 'F5.5', 'F6.3', 'F1.6', 'F5.6', 'F2.9',
       'F3.4', 'F4.3', 'F2.10', 'F1.7', 'F6.4', 'F4.4', 'F5.7', 'F3.5',
       'F2.11']
(survey
 >> gather("Answer","QrtSales", questions)
 >> mutate(likert = recode(X.QrtSales, likert_dict))
 >> drop(X.QrtSales)
 >> spread(X.person_ID, X.likert)
 >> head(100)
)

,Answer,1,2,3,4,5,6,7,8,9,...,255,256,257,258,259,260,261,262,263,264
0,F1,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0
1,F1.1,4.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,...,5.0,4.0,4.0,4.0,5.0,3.0,5.0,5.0,5.0,4.0
2,F1.2,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,...,5.0,4.0,4.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0
3,F1.3,4.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,...,5.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,5.0,4.0
4,F1.4,3.0,4.0,2.0,3.0,4.0,4.0,4.0,5.0,4.0,...,2.0,4.0,3.0,3.0,4.0,4.0,3.0,2.0,4.0,4.0
5,F1.5,4.0,3.0,5.0,4.0,4.0,5.0,4.0,5.0,4.0,...,5.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,5.0,4.0
6,F1.6,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,...,5.0,4.0,4.0,4.0,4.0,5.0,4.0,5.0,5.0,4.0
7,F1.7,4.0,4.0,4.0,3.0,4.0,4.0,3.0,4.0,5.0,...,5.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,4.0
8,F2,4.0,4.0,4.0,5.0,3.0,5.0,4.0,4.0,4.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0
9,F2.1,4.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0,4.0,...,4.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0


## Up Next

Stuff